In [30]:
# Imports

!pip install python-docx
!pip install docx

from docx import Document
from io import BytesIO
import re
import os
from pathlib import Path
import pickle
# from google.colab import files



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# RAG Mini Project
## Milestone #1 : Create and store Chunks
This notebook shows how to create text chunks from MS Word Documents.  
- Chunks all the word documents in a directory
- Uses python-docs to extract paragraph text for chunking
- Paragraphs are merged depending on parameterizable  max chunk size
- Document cleaning recommended for best results
-- remove diagrams and unnecessary text
-- merge paragraphs that are semantically similar

## Deliverables:
- Selection of multiple documents for your RAG project
- Capture chunks in a pickle file for next step (Embeddings)

When you generate the chunks with the size and semantics you want, then store them into a List and use Python's pickle library to save any Python data structure in a pickle file (.pkl) for later use and then recreate the data structure by loading the pickle file.

Deliverables:

- List of documents for your RAG project
- Jupyter Notebook
- Short summary of your efforts (issues, successes...)
- Pickle file with chunks for next step (Milestone 2)




### How to use pickle (good for Python data)
<code>import pickle<br>
my_list = ['apple', 'banana', 42, [1, 2, 3]]</code>

### Save the list
<code>with open('mylist.pkl', 'wb') as f:<br>
&nbsp;&nbsp;&nbsp;&nbsp; pickle.dump(my_list, f)
</code>
### Load the list
<code>with open('mylist.pkl', 'rb') as f:<br>
&nbsp;&nbsp;&nbsp;&nbsp; loaded_list = pickle.load(f)
</code>

In [24]:
import os
from docx import Document

# Define the directory where files will be saved
save_directory = "/home/javad/Downloads/INFO290/content"

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Creating a new set of documents on Bias in Machine Learning with multiple paragraphs
documents = {
    "1.BiasInMachineLearning.docx": [
        "Bias in machine learning (ML) has evolved alongside AI itself, reflecting the challenges of ensuring fairness in automated decision-making. "
        "Early AI systems inherited human biases due to limited representative data. This document traces the historical development of bias in AI systems, "
        "from early statistical models to deep learning and modern AI ethics concerns.",
        
        "Early statistical models (1950s-1980s) relied on manually curated datasets, often reflecting biases in data collection. "
        "Linear regression and early statistical learning approaches exhibited bias due to unrepresentative data.",
        
        "Machine learning growth (1990s-2010s) introduced more complex models, increasing reliance on historical data. "
        "Neural networks and decision trees amplified existing biases, leading to issues in credit scoring and facial recognition.",
        
        "The deep learning era (2010s-present) introduced large-scale AI models like GPT and DALL·E, demonstrating biases in language and image generation. "
        "Ethical concerns prompted fairness-aware algorithms and regulatory responses."
    ],

    "2.BiasInMachineLearning_CaseStudies.docx": [
        "AI bias has resulted in real-world consequences across multiple sectors. This document analyzes major case studies demonstrating the effects of bias in machine learning.",
        
        "Hiring and employment: Amazon's hiring algorithm (2018) showed gender bias, downgrading female candidates due to past hiring patterns. "
        "This case demonstrated how ML models can reinforce historical inequalities if trained on biased datasets.",
        
        "Law enforcement and criminal justice: The COMPAS algorithm used for criminal sentencing showed racial bias, falsely labeling Black defendants "
        "as high risk at twice the rate of white defendants. This sparked concerns over AI-driven legal discrimination.",
        
        "Healthcare disparities: A healthcare risk-prediction algorithm exhibited racial bias, allocating fewer resources to Black patients with similar "
        "health conditions as white patients. This highlights the risk of biased AI in critical decision-making areas.",
        
        "Financial sector: AI-driven credit scoring systems demonstrated bias against minority communities, leading to disproportionate loan denials. "
        "Such biases underscore the need for transparency in AI-based financial decisions."
    ],

    "3.RegulatoryApproaches_AI_Bias.docx": [
        "Governments worldwide are taking different approaches to regulate and mitigate bias in AI. This document explores key regulatory efforts.",
        
        "The European Union has introduced the AI Act (2021), which classifies AI systems based on risk levels. "
        "High-risk systems must meet strict fairness and transparency requirements.",
        
        "The United States lacks a comprehensive AI bias law but relies on the Federal Trade Commission (FTC) to enforce fairness "
        "under consumer protection laws. States like Illinois have passed biometric data regulations.",
        
        "China has implemented AI governance guidelines emphasizing fairness and transparency. Companies deploying AI must ensure non-discrimination in algorithms.",
        
        "Global organizations like UNESCO and OECD advocate for ethical AI principles focusing on non-discrimination, transparency, and fairness."
    ],

    "4.FairnessTechniquesInML.docx": [
        "Researchers have developed multiple techniques to reduce AI bias. This document outlines fairness-aware machine learning approaches.",
        
        "Pre-processing methods focus on data balancing techniques such as re-weighting samples and synthetic data generation to improve fairness before training.",
        
        "In-processing techniques include adversarial debiasing, where models are trained to minimize sensitive attribute influence, and regularization techniques "
        "that encourage fair decision boundaries.",
        
        "Post-processing techniques involve model auditing and explainability tools such as SHAP and LIME, which help interpret biased outcomes. "
        "Adjusting model predictions to meet fairness constraints is another key approach.",
        
        "Fairness-aware algorithms continue to evolve, integrating ethical considerations into AI development to mitigate bias and improve inclusivity."
    ],

    "5.Perspectives_EthicalAI.docx": [
        "AI fairness and ethics are global concerns. This document examines different international approaches to bias mitigation in AI.",
        
        "The United Nations advocates for human rights-based AI governance, emphasizing the importance of fairness and transparency in automated decision-making.",
        
        "The European Union enforces transparency and fairness in AI through GDPR and the AI Act, setting strict compliance standards for AI developers.",
        
        "The United States relies on industry standards and sector-specific regulations rather than a centralized AI fairness law. Federal agencies oversee AI ethics enforcement.",
        
        "China's government-led AI ethics guidelines focus on fairness, transparency, and security. These policies shape the development of ethical AI frameworks.",
        
        "The private sector, including companies like Google, IBM, and Microsoft, invests in AI fairness research, developing frameworks for responsible AI deployment."
    ]
}

file_paths = []
for file_name, parags in documents.items():
    doc = Document()
    for parag in parags:
        doc.add_paragraph(parag)  
    file_path = os.path.join(save_directory, file_name)
    doc.save(file_path)
    file_paths.append(file_path)

file_paths


['/home/javad/Downloads/INFO290/content/1.BiasInMachineLearning.docx',
 '/home/javad/Downloads/INFO290/content/2.BiasInMachineLearning_CaseStudies.docx',
 '/home/javad/Downloads/INFO290/content/3.RegulatoryApproaches_AI_Bias.docx',
 '/home/javad/Downloads/INFO290/content/4.FairnessTechniquesInML.docx',
 '/home/javad/Downloads/INFO290/content/5.Perspectives_EthicalAI.docx']

In [25]:
# Extract Chunks using document paragraphs
# Chunk size is controlled by parameter

def extract_fixed_chunks(file_path, chunk_size=1000):
    """
    Extract fixed-size chunks from a Word document.

    Args:
        file_path (str or bytes): Path to Word document or binary content
        chunk_size (int): Target size of each chunk in characters

    Returns:
        list: List of text chunks of approximately chunk_size characters
    """
    try:
        # Handle both file path and binary content
        if isinstance(file_path, bytes):
            doc = Document(BytesIO(file_path))
        else:
            doc = Document(file_path)

        # Extract and clean all text
        full_text = ""
        for para in doc.paragraphs:
            text = para.text.strip()
            if text:  # Skip empty paragraphs
                # Clean the text
                text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces
                full_text += text + " "  # Add space between paragraphs

        # Split text into sentences
        sentences = re.split('(?<=[.!?]) +', full_text)

        chunks = []
        current_chunk = ""

        for sentence in sentences:
            # If adding this sentence would exceed chunk_size
            if len(current_chunk) + len(sentence) > chunk_size:
                # If current chunk is not empty, add it to chunks
                if current_chunk:
                    chunks.append(current_chunk.strip())
                    current_chunk = ""

                # Handle sentences longer than chunk_size
                if len(sentence) > chunk_size:
                    # Split long sentence into fixed-size chunks
                    words = sentence.split()
                    temp_chunk = ""

                    for word in words:
                        if len(temp_chunk) + len(word) + 1 <= chunk_size:
                            temp_chunk += (" " + word if temp_chunk else word)
                        else:
                            chunks.append(temp_chunk.strip())
                            temp_chunk = word

                    if temp_chunk:
                        current_chunk = temp_chunk
                else:
                    current_chunk = sentence
            else:
                # Add sentence to current chunk
                current_chunk += (" " + sentence if current_chunk else sentence)

        # Add the last chunk if not empty
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks

    except Exception as e:
        raise Exception(f"Error processing document: {str(e)}")



In [33]:
# Main - Note that chunk size to use is set here in main and overrides default

def main():
    try:
        # Directory containing Word documents
        directory = "/home/javad/Downloads/INFO290/content"

        # Get all .docx files in the directory
        docx_files = list(Path(directory).glob("*.docx"))

        if not docx_files:
            print(f"No Word documents found in {directory}")
            return

        print(f"Found {len(docx_files)} Word documents")

        # Process each document
        for doc_path in docx_files:
            try:
                print(f"\nProcessing: {doc_path.name}")

                # Extract chunks of approximately 500 characters
                # MODIFY this as you see fit
                chunks = extract_fixed_chunks(str(doc_path), chunk_size=300)

                print(f"Created {len(chunks)} chunks")

                # Print first few chunks with their lengths
                for i, chunk in enumerate(chunks[:7], 1):
                    print(f"\nChunk {i} (length: {len(chunk)}):")
                    print(chunk)
                    print("-" * 50)

            except Exception as e:
                print(f"Error processing {doc_path.name}: {str(e)}")
                continue
        pickle_path = "/home/javad/Downloads/INFO290/content/chunks.pkl"
        with open(pickle_path, "wb") as f:
            pickle.dump(chunks, f)

        print(f"\nChunks saved to: {pickle_path}")

    except Exception as e:
        print(f"Error accessing directory: {str(e)}")

# Call main and start the chunking
main()



Found 5 Word documents

Processing: 1.BiasInMachineLearning.docx
Created 5 chunks

Chunk 1 (length: 215):
Bias in machine learning (ML) has evolved alongside AI itself, reflecting the challenges of ensuring fairness in automated decision-making. Early AI systems inherited human biases due to limited representative data.
--------------------------------------------------

Chunk 2 (length: 268):
This document traces the historical development of bias in AI systems, from early statistical models to deep learning and modern AI ethics concerns. Early statistical models (1950s-1980s) relied on manually curated datasets, often reflecting biases in data collection.
--------------------------------------------------

Chunk 3 (length: 214):
Linear regression and early statistical learning approaches exhibited bias due to unrepresentative data. Machine learning growth (1990s-2010s) introduced more complex models, increasing reliance on historical data.
--------------------------------------------

/content/sample_data/mydata